<a href="https://colab.research.google.com/github/carolribeiro95/NeuralLinkVisualIdentifier/blob/feature%2Fdataset/CNNDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aquisição das imagens

In [ ]:
%pip install bing-image-downloader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Download das imagens apenas em .jpg
from bing_image_downloader import downloader
downloader.download("dog", limit=100, output_dir='dataset',adult_filter_off=True,
    force_replace=False, timeout=60, verbose=True)
downloader.download("cat", limit=100, output_dir='dataset',adult_filter_off=True,
    force_replace=False, timeout=60, verbose=True)

In [ ]:
# Remove todos os arquivos com as extensões diferente de *.jpg
!find dataset -type f \( -name "*.webp" -o -name "*.png" -o -name "*.JPG" -o -name "*.jpeg" \) -delete

print("Limpeza concluída com sucesso.")

In [ ]:
# Importando bibliotecas

import pathlib
import PIL
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
url = '/content/dataset/'

In [ ]:
data_dir = pathlib.Path(url)

In [ ]:
# Contagem total de imagens no dataset
print(f'Total de imagens = {len(list(data_dir.rglob('*.jpg')))}')

In [ ]:
# Total de imagens de cat
print(f'Total de imagens de cat = {len(list(data_dir.glob('cat/*')))}')
# Total de imagens de dog
print(f'Total de imagens de dog = {len(list(data_dir.glob('dog/*')))}')

In [ ]:
subfolders = [f.name for f in data_dir.iterdir() if f.is_dir()]
subfolders

In [ ]:
# visualização de imagens de dog training
dog = list(data_dir.glob('dog/*'))
PIL.Image.open(str(dog[0]))




In [ ]:
# Carregando imagem de um cat
cat = list(data_dir.glob('cat/*'))
PIL.Image.open(str(cat[0]))

In [ ]:
# Percorre todas as imagens e verifica a dimensão
def print_dimensoes_imagem(diretorio):
  for i in range(len(list(diretorio.rglob('*.jpg')))):
    img = PIL.Image.open(list(diretorio.rglob('*.jpg'))[i])
    img_array = np.array(img)
    print(f'Dimensão da imagem {i} = {img_array.shape}')


print_dimensoes_imagem(data_dir)


In [ ]:
# Redimensionamento das imagens para 256 x 256
def redimensionar_imagem(diretorio_path, tamanho=(256, 256)):
    pasta = pathlib.Path(diretorio_path)
    arquivos = list(pasta.rglob('*.jpg'))

    print(f"Iniciando redimensionamento de {len(arquivos)} imagens...")

    for i, caminho_img in enumerate(arquivos):
        try:

            with PIL.Image.open(caminho_img) as img:
                # Converte para RGB (garante compatibilidade total)
                # Usa o filtro LANCZOS para manter a nitidez dos detalhes
                img_redimensionada = img.convert("RGB").resize(tamanho,
                PIL.Image.Resampling.LANCZOS)

                # 5. Sobrescreve o arquivo original
                img_redimensionada.save(caminho_img, "JPEG", quality=90)

            if i % 100 == 0: # Printa a cada 100 imagens para não poluir o terminal
                print(f"{i} imagens processadas...")

        except Exception as e:
            print(f"Erro ao processar {caminho_img.name}: {e}")

    print("Finalizado com sucesso!")



In [ ]:

redimensionar_imagem(url)

In [ ]:
# Print total de imagens de dog redimensionadas
print(f'Total de imagens de dog redimensionadas = {len(list(data_dir.glob("dog/*")))}')
# Print total de imagens de cat redimensionadas
print(f'Total de imagens de cat redimensionadas = {len(list(data_dir.glob("cat/*")))}')

In [ ]:
print_dimensoes_imagem(data_dir)

In [ ]:
batch_size = 64
altura = 256
largura = 256

Divisão da Base de Dados

In [ ]:
# Carregamento do Dataset de treinamento

treino = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(altura, largura),
    batch_size=batch_size
)

In [ ]:
validacao = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(altura, largura),
    batch_size=batch_size
)

In [ ]:
classe_saida = treino.class_names

In [ ]:
tf.random.set_seed(123)

In [ ]:
# Definição das camadas da rede

modelo = tf.keras.models.Sequential([
   # Camada de entrada (Input layer for Sequential model)
    tf.keras.layers.Input(shape=(largura, altura, 3)),
    # Escalar valores de 0 a 1
    tf.keras.layers.Rescaling(1./255),
   # Camadas convolucionais
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
   # Quebrar a imagem para uma sequência de números
    tf.keras.layers.Flatten(),
   # Camada escondida
    tf.keras.layers.Dense(128, activation='relu'),
   # Camada de saída
    tf.keras.layers.Dense(len(classe_saida), activation='softmax')
])

In [ ]:
modelo.compile(
    optimizer='adam',
    loss= 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
# Treinamento do modelo
history = modelo.fit(
    treino,
    validation_data=validacao,
    epochs=10
)

In [ ]:
modelo.summary()

In [ ]:
# Função para plot de gráfico acurácia treino x validação
def plota_resultados(history,epocas):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  intervalo_epocas = range(epocas)

  plt.figure(figsize=(12,6))
  plt.subplot(1,2,1)
  plt.plot(intervalo_epocas,acc,'r',label='Acurácia do Treino')
  plt.plot(intervalo_epocas,val_acc,'b',label='Acurácia da Validação')
  plt.legend(loc='lower right')

  plt.subplot(1,2,2)

  plt.plot(intervalo_epocas,loss,'r',label='Perda do Treino')
  plt.plot(intervalo_epocas,val_loss,'b',label='Perda da Validação')
  plt.legend(loc='upper right')
  plt.show()

In [ ]:
plota_resultados(history,10)

Técnicas para evitar Overfitting do modelo


In [ ]:
# Data Augmentation

data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.05)
])
#


In [ ]:
# Modelo 2 - Técnicas para controle de overfitting: Data Augmentatio e Dropout

modelo_2 = tf.keras.models.Sequential([
    # Camada de entrada (Input layer for Sequential model)
    tf.keras.layers.Input(shape=(largura, altura, 3)),
    # Data Augmentation
    data_augmentation,
    # Escalar valores de 0 a 1
    tf.keras.layers.Rescaling(1./255),
   # Camadas convolucionais
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Dropout
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
   # Quebrar a imagem para uma sequência de números
    tf.keras.layers.Flatten(),
   # Dropout
    tf.keras.layers.Dropout(0.5),
   # Camada escondida
    tf.keras.layers.Dense(128, activation='relu'),
   # Camada de saída
    tf.keras.layers.Dense(len(classe_saida), activation='softmax')
])


In [ ]:
modelo_2.compile(
    optimizer='adam',
    loss= 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
# Aumentar o número de épocas para 30
epocas = 30
history = modelo_2.fit(
    treino,
    validation_data=validacao,
    epochs=epocas
)

In [ ]:
modelo_2.summary()

In [ ]:
plota_resultados(history,epocas)